In [ ]:
# 
# This Notebook requires the following environment variables (.env file):
# 

# CLEAN_AUDIO_LOCAL_WORKING_DIR=TODO
# CLEAN_AUDIO_INPUT_CONTAINER_SAS_URI=TODO
# CLEAN_AUDIO_OUTPUT_CONTAINER_SAS_URI=TODO

In [ ]:
import os
import shutil
import subprocess
from dotenv import load_dotenv
from utils import get_container_client_from_sas, download_blobs, upload_files

#
# Main
#

load_dotenv()

LOCAL_WORKING_DIR = os.getenv("CLEAN_AUDIO_LOCAL_WORKING_DIR")
INPUT_CONTAINER_SAS_URI = os.getenv("CLEAN_AUDIO_INPUT_CONTAINER_SAS_URI")
OUTPUT_CONTAINER_SAS_URI = os.getenv("CLEAN_AUDIO_OUTPUT_CONTAINER_SAS_URI")

print("Starting clean audio process...")

input_folder = os.path.join("LOCAL_WORKING_DIR", "clean-audio-input")
output_folder = os.path.join("LOCAL_WORKING_DIR", "clean-audio-output")

os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

input_container_client = get_container_client_from_sas(INPUT_CONTAINER_SAS_URI)
output_container_client = get_container_client_from_sas(OUTPUT_CONTAINER_SAS_URI)

print("Downloading blobs from input container...")
audio_files = download_blobs(input_container_client, local_folder=input_folder, suffix=".mp3")

print(f"Convert {len(audio_files)} audio files to clean audio...")

clean_files = []
for audio_file in audio_files:
    subprocess.run([
        "demucs",
        "--two-stems", "vocals",
        "--out", output_folder,
        audio_file.local_path
    ], check=True)

    clean_files.append(os.path.join(output_folder, audio_file.blob_name))

if clean_files:
    uploaded_files = upload_files(output_container_client, clean_files)
    print(f"Uploaded {len(uploaded_files)} clean audio files to output container.")

shutil.rmtree(input_folder)
shutil.rmtree(output_folder)

print("Clean audio process completed.")